# 🤖 Xây Dựng Agent Đầu Tiên với Google ADK

<a href="https://colab.research.google.com/github/chitoan1992/devfest-google-adk/blob/master/Building_your_first_Agent_vn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 10 năm trước: AI agent chỉ trong phim viễn tưởng
> 
> 5 năm trước: Chỉ trong labs nghiên cứu
> 
> **Hôm nay: Bạn tự build trong 90 phút**

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/LinkToWorkshop.png)

## ADK là gì?

**Agent Development Kit (ADK)** - Framework Python giúp bạn xây dựng ứng dụng AI với LLMs một cách đơn giản và mạnh mẽ.

![](https://google.github.io/adk-docs/assets/adk-components.png)

> **"Ai đã từng làm việc với LangChain hay AutoGen? Điểm khác biệt của ADK là gì?"**

**Bạn sẽ học được gì:**
- ✨ Xây dựng agent đầu tiên với tools (Weather Bot)
- 👥 Thiết kế multi-agent teams với automatic delegation
- 🚀 Setup fullstack agent app với ADK Starter Pack
- 📋 Áp dụng agents cho hackathon workflow (MVP Planning + Pitch Generator)
- ⚡ Sử dụng AI builders (AI Studio, WebSim) để tạo app nhanh chóng

**Yêu cầu:**
- ✅ Python cơ bản
- ✅ Hiểu về LLMs & APIs
- ✅ API Keys (Google AI Studio)

![](https://google.github.io/adk-docs/assets/adk-lifecycle.png)

## 📋 Bước 0: Cài đặt

### Lấy Gemini API Credit
👉 **Link nhận credit:** https://trygcp.dev/claim/gdg-mientrung-road-to-devfest-code-1

👉 **Link slide hướng dẫn:** [Hướng dẫn lấy credit](https://docs.google.com/presentation/d/1fnQNauWcxgt5eqhTAuaBGn2giNQU6f_f_fv3R7nAvp4/edit?slide=id.g2399cbc1e14_0_411&resourcekey=0-dlnAU4LYN8QV27uVMLeIQA#slide=id.g2399cbc1e14_0_411)

✅ **Checkpoint:** Bạn có API key? Copy và paste vào cell dưới.

In [ ]:
# @title Cấu hình API Keys (Thay thế bằng keys thực tế của bạn!)
import os
from google.colab import userdata


# Gemini API Key (Lấy từ Google AI Studio: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Cấu hình ADK để sử dụng API keys trực tiếp
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# @markdown **Lưu ý Bảo mật:** Nên quản lý API keys một cách an toàn (ví dụ: sử dụng Colab Secrets hoặc environment variables) thay vì hardcode trực tiếp trong notebook. Thay thế các placeholder strings ở trên.

In [ ]:
# @title Thiết lập và Cài đặt
# Cài đặt Google ADK

!pip install -U -q google-adk

In [2]:
# Kiểm tra phiên bản ADK
from google import adk

print(adk.__version__)

1.16.0


### Khởi động lại runtime

Để sử dụng các packages vừa cài đặt, cần khởi động lại runtime.

**Lưu ý:** Sau khi restart, chạy lại các cells từ đầu (bỏ qua cell restart này).

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [3]:
# @title Import các thư viện cần thiết
import datetime
import os
from zoneinfo import ZoneInfo
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

import warnings
warnings.filterwarnings("ignore")

import logging
# Tắt tất cả logging để output sạch hơn
logging.basicConfig(level=logging.CRITICAL)
logging.getLogger('asyncio').setLevel(logging.CRITICAL)
logging.getLogger('google_genai').setLevel(logging.CRITICAL)
logging.getLogger('google.adk').setLevel(logging.CRITICAL)

# Thêm colored output cho Colab
from IPython.display import HTML, display

class Colors:
    """ANSI color codes cho terminal output"""
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def print_success(text):
    """In text màu xanh lá"""
    print(f"{Colors.OKGREEN}{text}{Colors.ENDC}")

def print_info(text):
    """In text màu cyan"""
    print(f"{Colors.OKCYAN}{text}{Colors.ENDC}")

def print_warning(text):
    """In text màu vàng"""
    print(f"{Colors.WARNING}{text}{Colors.ENDC}")

def print_error(text):
    """In text màu đỏ"""
    print(f"{Colors.FAIL}{text}{Colors.ENDC}")

def print_header(text):
    """In header màu tím bold"""
    print(f"{Colors.BOLD}{Colors.HEADER}{text}{Colors.ENDC}")

print_success("✅ Đã import thành công các thư viện ADK!")

✅ Đã import thành công các thư viện ADK!


In [ ]:
# --- Định nghĩa Model Constants ---
AGENT_MODEL = "gemini-2.5-flash"

print_info(f"🚀 Sẵn sàng xây dựng agent với {Colors.BOLD}Gemini 2.5 Flash{Colors.ENDC}!")

---

> **"Giới hạn của mô hình ngôn ngữ lớn là gì ?"**

## 🌤️ Bước 1: Weather Agent - Agent Đầu Tiên

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/Weather%20Agent.svg)

Chúng ta sẽ xây dựng Weather Bot với 2 thành phần cốt lõi:

### 1. 🔧 Tool: Khả Năng Hành Động

**Tool** = Python function trang bị khả năng cho agent (gọi API, query database, tính toán...)

**💡 Bí quyết:** Docstring chính là "sách hướng dẫn" cho LLM hiểu tool:
- Làm **gì**?
- **Khi nào** dùng?
- **Tham số** nào?
- **Trả về** gì?

> **"Tại sao chúng ta lại cần TOOLS?"**

### 2. 🧠 Agent: Bộ Não AI

**Agent** = LLM + Instructions + Tools
- Hiểu yêu cầu của user
- Quyết định khi nào & cách sử dụng tools
- Tạo phản hồi thông minh

> **"Nếu tool return một string dài 1000 chữ về thời tiết, agent có xử lý được không?"**


In [ ]:
# @title Tool lấy thời tiết

def get_weather(city: str) -> dict:
    """Lấy báo cáo thời tiết hiện tại cho một thành phố.

    Args:
        city: Tên thành phố (VD: "Hà Nội", "London", "Tokyo")

    Returns:
        dict: {'status': 'success'|'error', 'report': str} hoặc {'error_message': str}
    """
    print_info(f"🔍 Đang tra cứu thời tiết: {Colors.BOLD}{city}{Colors.ENDC}")

    city_normalized = city.lower().replace(" ", "")

    # Mock weather database
    mock_weather_db = {
        "hanoi": {"status": "success", "report": "☀️ Hà Nội: Nắng đẹp, 25°C"},
        "hànội": {"status": "success", "report": "☀️ Hà Nội: Nắng đẹp, 25°C"},
        "danang": {"status": "success", "report": "☀️ Đà Nẵng: Nắng đẹp, 27°C"},
        "đànẵng": {"status": "success", "report": "☀️ Đà Nẵng: Nắng đẹp, 27°C"},
        "london": {"status": "success", "report": "☁️ London: Nhiều mây, 15°C"},
        "tokyo": {"status": "success", "report": "🌧️ Tokyo: Mưa nhẹ, 18°C"},
    }

    if city_normalized in mock_weather_db:
        result = mock_weather_db[city_normalized]
        print_success(f"✅ Tìm thấy dữ liệu thời tiết!")
        return result
    else:
        print_error(f"❌ Không có dữ liệu cho '{city}'")
        return {"status": "error", "error_message": f"Xin lỗi, không có thông tin thời tiết cho '{city}'."}

# Test tool
print_header("\n📝 Test tool:")
print(get_weather("Hà Nội"))
print(get_weather("Đà Nẵng"))
print(get_weather("Unknown Location"))


📝 Test tool:
🔍 Đang tra cứu thời tiết: Hà Nội
✅ Tìm thấy dữ liệu thời tiết!
{'status': 'success', 'report': '☀️ Hà Nội: Nắng đẹp, 25°C'}
🔍 Đang tra cứu thời tiết: Đà Nẵng
✅ Tìm thấy dữ liệu thời tiết!
{'status': 'success', 'report': '☀️ Đà Nẵng: Nắng đẹp, 27°C'}
🔍 Đang tra cứu thời tiết: Unknown Location
❌ Không có dữ liệu cho 'Unknown Location'
{'status': 'error', 'error_message': "Xin lỗi, không có thông tin thời tiết cho 'Unknown Location'."}


### 💪 **Coding Challenge!**

**Nhiệm vụ:** Nâng cấp hàm `get_weather()` để sử dụng API thời tiết thực tế thay vì mock data.

**API:** `https://goweather.xyz/weather/{city_normalized}`

**Ví dụ:** https://goweather.xyz/weather/Danang

**Gợi ý:**
- Sử dụng `requests` hoặc `httpx` để gọi API
- Parse JSON response
- Xử lý error cases (API fail, city not found...)


### 🎯 Tạo Weather Agent

**Công thức Agent:**
```
Agent = Model + Instructions + Tools + Description
```

**Các thành phần quan trọng:**
- `name`: ID duy nhất
- `model`: LLM nào được dùng (Gemini, GPT, Claude...)
- `description`: Tóm tắt ngắn gọn về agent (dùng cho delegation)
- `instruction`: Hướng dẫn chi tiết cách hoạt động
- `tools`: Danh sách Python functions

In [ ]:
# @title Định nghĩa Weather Agent
weather_agent = Agent(
    name="weather_agent",
    model=AGENT_MODEL,
    description="Cung cấp thông tin thời tiết cho các thành phố.",
    instruction="Bạn là trợ lý thời tiết. "
                "Khi user hỏi về thời tiết, dùng tool 'get_weather'. "
                "Nếu tool lỗi, thông báo lịch sự. "
                "Nếu thành công, trình bày báo cáo rõ ràng.",
    tools=[get_weather],
)

print_success(f"✅ Agent '{Colors.BOLD}{weather_agent.name}{Colors.ENDC}' đã sẵn sàng với model {Colors.OKCYAN}{AGENT_MODEL}{Colors.ENDC}!")

✅ Agent 'weather_agent' đã sẵn sàng với model gemini-2.5-flash!


---

> **"Nếu tôi muốn agent nhớ preferences của user (ví dụ: luôn trả lời bằng tiếng Anh), làm thế nào?"**

### ⚙️ Thiết lập Runner & Session

**2 components để chạy agent:**

#### 1. 💾 SessionService
Quản lý conversation history & state
- `InMemorySessionService`: Lưu trong RAM (cho demo/testing)

#### 2. 🎬 Runner
Engine điều phối tương tác
- Nhận input từ user
- Định tuyến đến agent
- Quản lý LLM & tool calls
- Yield events theo tiến trình

In [ ]:
# @title Thiết lập Session Service và Runner

# Session Service: Quản lý conversation history
session_service = InMemorySessionService()

# Định nghĩa context
APP_NAME = "Weather Agent at GDG DevFest Đà Nẵng 2025"
USER_ID = "user_1"
SESSION_ID = "session_001"

# Tạo session
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print_success(f"✅ Session: {Colors.BOLD}{SESSION_ID}{Colors.ENDC}")

# Runner: Điều phối execution
runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service
)
print_info(f"🎬 Runner sẵn sàng cho agent '{Colors.BOLD}{runner.agent.name}{Colors.ENDC}'")

✅ Session: session_001
🎬 Runner sẵn sàng cho agent 'weather_agent'


---

### 💬 Tương Tác Với Agent

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/Weather%20Agent%20Flow.svg)

**Async Flow:**
1. Gửi query → `runner.run_async()`
2. Runner yield **Events** (tool calls, LLM thoughts, responses...)
3. Tìm `final_response` event
4. Hiển thị kết quả

**Tại sao async?** LLM & API calls là I/O-bound → async = hiệu quả hơn!

In [7]:
# @title Định nghĩa Agent Interaction Function

async def call_agent_async(query: str, runner, user_id, session_id):
    """Gửi query đến agent và hiển thị response."""
    # Header với màu
    print(f"\n{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}")
    print(f"{Colors.BOLD}👤 User:{Colors.ENDC} {Colors.OKBLUE}{query}{Colors.ENDC}")
    print(f"{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}")

    # Chuẩn bị message
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "⚠️ Agent không tạo ra response."

    # Thực thi agent và lặp qua events
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate:
                final_response_text = f"⚠️ Agent escalated: {event.error_message or 'Không có message.'}"
            break

    # Response với màu
    print(f"\n{Colors.BOLD}🤖 Agent:{Colors.ENDC} {Colors.OKGREEN}{final_response_text}{Colors.ENDC}")
    print(f"{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}\n")

async def call_agent_streaming(query: str, runner, user_id, session_id):
    """Gửi query đến agent và hiển thị response với streaming events."""
    # Header với màu
    print(f"\n{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}")
    print(f"{Colors.BOLD}👤 User:{Colors.ENDC} {Colors.OKBLUE}{query}{Colors.ENDC}")
    print(f"{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}\n")

    # Chuẩn bị message
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "⚠️ Agent không tạo ra response."

    # Thực thi agent và lặp qua events với streaming
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # 1. Hiển thị delegation events
        if hasattr(event, 'delegated_to') and event.delegated_to:
            print_warning(f"🔄 Delegating to: {Colors.BOLD}{event.delegated_to}{Colors.ENDC}")

        # 2. Hiển thị tool calls
        if hasattr(event, 'tool_calls') and event.tool_calls:
            for tool_call in event.tool_calls:
                tool_name = tool_call.name if hasattr(tool_call, 'name') else 'unknown'
                print_info(f"🔧 Tool called: {Colors.BOLD}{tool_name}{Colors.ENDC}")

        # 3. Hiển thị thinking/reasoning (nếu có)
        if hasattr(event, 'thought') and event.thought:
            print(f"{Colors.WARNING}💭 Thinking: {event.thought}{Colors.ENDC}")

        # 4. Streaming response content
        if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text and not event.is_final_response():
                    # Stream partial responses
                    print(f"{Colors.OKGREEN}{part.text}{Colors.ENDC}", end="", flush=True)

        # 5. Final response
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response_text = event.content.parts[0].text
                print(f"\n\n{Colors.BOLD}🤖 Agent (Final):{Colors.ENDC} {Colors.OKGREEN}{final_response_text}{Colors.ENDC}")
            elif event.actions and event.actions.escalate:
                final_response_text = f"⚠️ Agent escalated: {event.error_message or 'Không có message.'}"
                print(f"\n{Colors.FAIL}{final_response_text}{Colors.ENDC}")
            break

    print(f"\n{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}\n")



---

### 🎮 Test Agent!

In [ ]:
# @title Chạy Conversation

async def run_conversation():
    print_header("\n🎬 BẮT ĐẦU CONVERSATION")

    await call_agent_async("Thời tiết ở Hà Nội như thế nào?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Còn Paris thì sao?",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Cho tôi biết thời tiết ở London",
                           runner=runner,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    print_header("🏁 KẾT THÚC CONVERSATION\n")

await run_conversation()


🎬 BẮT ĐẦU CONVERSATION

👤 User: Thời tiết ở Hà Nội như thế nào?
🔍 Đang tra cứu thời tiết thực tế: Hà Nội
❌ Lỗi khi gọi API thời tiết: HTTPSConnectionPool(host='goweather.xyz', port=443): Read timed out. (read timeout=10)

🤖 Agent: Xin lỗi, tôi không thể lấy thông tin thời tiết cho Hà Nội vào lúc này. Vui lòng thử lại sau.


👤 User: Còn Paris thì sao?
🔍 Đang tra cứu thời tiết thực tế: Paris
❌ Lỗi khi gọi API thời tiết: 404 Client Error: Not Found for url: https://goweather.xyz/weather/paris

🤖 Agent: Xin lỗi, tôi không thể lấy thông tin thời tiết cho Paris vào lúc này. Vui lòng thử lại sau.


👤 User: Cho tôi biết thời tiết ở London
🔍 Đang tra cứu thời tiết thực tế: London
❌ Lỗi khi gọi API thời tiết: 404 Client Error: Not Found for url: https://goweather.xyz/weather/london

🤖 Agent: Xin lỗi, tôi không thể lấy thông tin thời tiết cho London vào lúc này. Vui lòng thử lại sau.

🏁 KẾT THÚC CONVERSATION



---

## 🎉 Hoàn Thành Bước 1!

**Bạn đã tạo thành công agent đầu tiên:**
- ✅ Hiểu yêu cầu của user
- ✅ Sử dụng tool để tìm thông tin
- ✅ Phản hồi thông minh dựa trên kết quả

**Tiếp theo:** Xây dựng multi-agent team! 👥

## 👥 Bước 2: Multi-Agent Team - Delegation

### Tại Sao Cần Agent Team?

**Vấn đề:** 1 agent làm mọi thứ → khó quản lý, kém hiệu quả

**Giải pháp:** Specialized Agent Team
- 🧩 **Modularity:** Dễ develop, test, maintain
- 🎯 **Specialization:** Mỗi agent tối ưu cho 1 task
- 📈 **Scalability:** Thêm khả năng = thêm agent
- ⚡ **Efficiency:** Dùng model phù hợp cho từng task

### Kiến Trúc Agent Team

```
Root Agent (Orchestrator)
    ├── 👋 Greeting Agent
    ├── 🌤️ Weather Agent  
    └── 👋 Farewell Agent
```

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/Multi%20Agent%20Team.svg)

**Automatic Delegation:** Root agent tự động chuyển task đến sub-agent phù hợp nhất dựa trên `description`

> **"Nếu user hỏi: 'Xin chào! Thời tiết Hà Nội thế nào?' - agent xử lý thế nào?"**

---

### 🔧 Tools Cho Sub-Agents

In [ ]:
# @title Định nghĩa Tools cho Greeting và Farewell
from typing import Optional

def say_hello(name: Optional[str] = None) -> str:
    """Cung cấp lời chào thân thiện.

    Args:
        name: Tên người cần chào (optional)

    Returns:
        str: Message chào hỏi
    """
    if name:
        greeting = f"👋 Xin chào, {name}!"
        print_info(f"🔔 say_hello được gọi với: {Colors.BOLD}{name}{Colors.ENDC}")
    else:
        greeting = "👋 Xin chào!"
        print_info(f"🔔 say_hello được gọi (không có tên)")
    return greeting

def say_goodbye() -> str:
    """Cung cấp message tạm biệt."""
    print_info(f"🔔 say_goodbye được gọi")
    return "👋 Tạm biệt! Chúc bạn một ngày tốt lành! ✨"

# Test
print_header("📝 Test tools:")
print(say_hello("Minh"))
print(say_hello())
print(say_goodbye())

📝 Test tools:
🔔 say_hello được gọi với: Minh
👋 Xin chào, Minh!
🔔 say_hello được gọi (không có tên)
👋 Xin chào!
🔔 say_goodbye được gọi
👋 Tạm biệt! Chúc bạn một ngày tốt lành! ✨


---

### 👋 Tạo Sub-Agents

**💡 Chìa khóa delegation:** `description` phải rõ ràng, ngắn gọn!

In [ ]:
# @title Định nghĩa Sub-Agents

# Greeting Agent
greeting_agent = Agent(
    model=AGENT_MODEL,
    name="greeting_agent",
    instruction="Nhiệm vụ DUY NHẤT: Chào hỏi thân thiện. "
                "Dùng tool 'say_hello'. "
                "Nếu user cung cấp tên, truyền cho tool. "
                "KHÔNG làm gì khác.",
    description="Xử lý greetings sử dụng 'say_hello'.",
    tools=[say_hello],
)
print_success(f"✅ {Colors.BOLD}{greeting_agent.name}{Colors.ENDC} sẵn sàng!")

# Farewell Agent
farewell_agent = Agent(
    model=AGENT_MODEL,
    name="farewell_agent",
    instruction="Nhiệm vụ DUY NHẤT: Tạm biệt lịch sự. "
                "Dùng tool 'say_goodbye' khi user nói bye/tạm biệt. "
                "KHÔNG làm gì khác.",
    description="Xử lý farewells sử dụng 'say_goodbye'.",
    tools=[say_goodbye],
)
print_success(f"✅ {Colors.BOLD}{farewell_agent.name}{Colors.ENDC} sẵn sàng!")

✅ greeting_agent sẵn sàng!
✅ farewell_agent sẵn sàng!


---

### 🎯 Root Agent với Sub-Agents

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/Root%20and%20SubAgent.svg)

**Automatic Delegation Magic:**
- Root agent nhận query
- LLM xem `description` của các sub-agents
- Nếu match → tự động delegate
- Sub-agent xử lý với tools riêng

> **"Nếu có 10 sub-agents, performance có bị ảnh hưởng không?"**


In [ ]:
# @title Định nghĩa Root Agent với Sub-Agents

weather_agent_team = Agent(
    name="weather_agent_v2",
    model=AGENT_MODEL,
    description="Agent điều phối: Weather requests + delegate greetings/farewells.",
    instruction="Bạn là Weather Agent chính điều phối team. "
                "TRÁCH NHIỆM CHÍNH: Cung cấp thông tin thời tiết. "
                "Dùng 'get_weather' CHỈ cho weather requests. "
                "\n\nSUB-AGENTS: "
                "1. 'greeting_agent': Xử lý 'Hi', 'Xin chào' → DELEGATE "
                "2. 'farewell_agent': Xử lý 'Tạm biệt', 'Bye' → DELEGATE "
                "\n\nPHÂN TÍCH query: "
                "- Greeting? → delegate greeting_agent "
                "- Farewell? → delegate farewell_agent "
                "- Weather? → tự xử lý với get_weather "
                "- Khác? → phản hồi lịch sự hoặc nói không xử lý được",
    tools=[get_weather],
    sub_agents=[greeting_agent, farewell_agent]
)

print_success(f"✅ Root Agent '{Colors.BOLD}{weather_agent_team.name}{Colors.ENDC}' với sub-agents:")
for sa in weather_agent_team.sub_agents:
    print_info(f"   └─ {sa.name}")

✅ Root Agent 'weather_agent_v2' với sub-agents:
   └─ greeting_agent
   └─ farewell_agent


---

### 🎮 Test Agent Team Delegation

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/Multi%20Agent%20Team%20Flow.svg)

**Expected Flow:**
1. "Xin chào!" → `greeting_agent` → `say_hello()`
2. "Thời tiết Hà Nội?" → `weather_agent_v2` → `get_weather()`
3. "Tạm biệt!" → `farewell_agent` → `say_goodbye()`

> **"Nếu user hỏi: 'Xin chào! Thời tiết Hà Nội thế nào?' - agent xử lý thế nào?"**

In [ ]:
# @title Test Agent Team

async def run_team_conversation():
    print_header("\n🎬 TESTING AGENT TEAM DELEGATION")
    print(f"{Colors.BOLD}{Colors.OKCYAN}{'='*60}{Colors.ENDC}")

    # Setup riêng cho test này
    session_service = InMemorySessionService()
    APP_NAME = "agent_team_demo"
    USER_ID = "user_team"
    SESSION_ID = "session_team_001"

    session = await session_service.create_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
    )
    print_success(f"✅ Session: {Colors.BOLD}{SESSION_ID}{Colors.ENDC}\n")

    runner_team = Runner(
        agent=weather_agent_team,
        app_name=APP_NAME,
        session_service=session_service
    )

    # Test delegation
    await call_agent_async("Xin chào!",
                           runner=runner_team,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Thời tiết ở Đà Nẵng thế nào?",
                           runner=runner_team,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    await call_agent_async("Cảm ơn, tạm biệt!",
                           runner=runner_team,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

    print_header("🏁 KẾT THÚC TEST\n")

await run_team_conversation()

### 🔍 Phân Tích Kết Quả

**Quan sát logs để thấy delegation:**
- ✅ "Xin chào!" → `say_hello` được gọi
- ✅ "Thời tiết Hà Nội?" → `get_weather` được gọi
- ✅ "Tạm biệt!" → `say_goodbye` được gọi

**Automatic delegation thành công!** Root agent thông minh định tuyến đúng request đến đúng specialist! 🎯

---

## 🚀 Bước 3: ADK Starter Pack - Xây Dựng Fullstack Agent App

### Tại Sao Cần ADK Starter Pack?

**Vấn đề:** Bạn đã biết cách build agents, nhưng:
- ❌ Phải setup project từ đầu (structure, config, dependencies...)
- ❌ Chưa biết cách integrate frontend với ADK backend
- ❌ Mất thời gian research best practices

**Giải pháp:** ADK Starter Pack
- ✅ Template project ready-to-use
- ✅ Fullstack architecture được optimize
- ✅ Pre-configured agents & tools
- ✅ Modern UI (React/Vue)
- ✅ Production-ready setup

### ADK Starter Pack là gì?

**Agent Starter Pack** = Boilerplate project cho agent applications với:
- 🏗️ **Backend:** ADK agents với session management
- 🎨 **Frontend:** Modern web UI (React/Vite)
- 🔧 **Tools:** Pre-configured utilities
- 📦 **DevOps:** Docker, Make commands, deployment configs
- 🚀 **Examples:** Sample agents để học

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/Agent%20Starter%20Pack.svg)

---

### 📦 Bước 1: Cài đặt Agent Starter Pack

**Có 2 cách cài đặt:**

#### Option 1: Sử dụng uvx (Khuyến nghị)
```bash
uvx agent-starter-pack create my-agent
```

#### Option 2: Clone từ GitHub
```bash
git clone https://github.com/GoogleCloudPlatform/agent-starter-pack
cd agent-starter-pack
```

**Chọn template:** `gemini-fullstack`

---

In [9]:
# @title Cài đặt uv package manager

!pip install -q uv

print_success("✅ Đã cài đặt uv!")
print_info("💡 Bước tiếp theo:")
print("   1. Mở terminal: Cmd/Ctrl + Shift + P → 'Terminal'")
print("   2. Chạy: uvx agent-starter-pack create my-agent")
print("   3. Chọn: gemini-fullstack")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.0 MB/s eta 0:00:00
✅ Đã cài đặt uv!
💡 Bước tiếp theo:
   1. Mở terminal: Cmd/Ctrl + Shift + P → 'Terminal'
   2. Chạy: uvx agent-starter-pack create my-agent
   3. Chọn: gemini-fullstack


### ⚙️ Bước 2: Cấu hình Environment

**Tạo file `.env` trong thư mục project:**

```bash
cd my-agent/app
```

**Nội dung file `.env`:**
```env
GOOGLE_GENAI_USE_VERTEXAI=0
GOOGLE_API_KEY=your_api_key_here
```

**💡 Lưu ý:** Thay `your_api_key_here` bằng API key thực tế từ [Google AI Studio](https://aistudio.google.com/app/apikey)

---

In [12]:
# @title (Colab Only) Tạo file .env tự động

from google.colab import userdata

# Lấy API Key từ Colab userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Ghi nội dung vào file .env
with open('my-agent/app/.env', 'w') as f:
    f.write(f"GOOGLE_GENAI_USE_VERTEXAI=0\n")
    f.write(f"GOOGLE_API_KEY={api_key}\n")

print_success("✅ Đã tạo và cấu hình file my-agent/app/.env!")

✅ Đã tạo và cấu hình file my-agent/app/.env!


### 🚀 Bước 3: Chạy Application

**Cài đặt dependencies và chạy:**

```bash
cd gemini-fullstack
make install
make dev
```

**Giải thích commands:**
- `make install`: Cài đặt Python & Node dependencies
- `make dev`: Chạy backend (port 8000) + frontend (port 5173)

**Kiểm tra:**
- Backend: http://localhost:8000
- Frontend: http://localhost:5173
- API Docs: http://localhost:8000/docs

---

In [ ]:
# @title (Colab Only) Hiển thị Web UI trong Colab

from google.colab import output

print_header("\n🌐 Khởi động Web UI...")
print_info("Agent web interface sẽ hiển thị bên dưới:")

# Hiển thị web UI (port 5173 cho Vite frontend)
output.serve_kernel_port_as_iframe(5173)

print_success("\n✅ Web UI đã sẵn sàng!")
print_info("💡 Tips:")
print("   - UI sẽ load sau khi chạy 'make dev' trong terminal")
print("   - Chat với agent để test functionality")
print("   - Xem console logs để debug")

# Nếu muốn public URL (optional)
# output.serve_kernel_port_as_window(5173)


🌐 Khởi động Web UI...
Agent web interface sẽ hiển thị bên dưới:


<IPython.core.display.Javascript object>


✅ Web UI đã sẵn sàng!
💡 Tips:
   - UI sẽ load sau khi chạy 'make dev' trong terminal
   - Chat với agent để test functionality
   - Xem console logs để debug


### 🎯 Bước 4: Explore Project Structure

**Cấu trúc thư mục quan trọng:**

```
gemini-fullstack/
├── app/
│   ├── agents/          # Agent definitions
│   ├── tools/           # Tool functions
│   ├── main.py          # FastAPI backend
│   └── .env             # Environment config
├── frontend/
│   ├── src/
│   │   ├── components/  # React components
│   │   └── App.tsx      # Main app
│   └── package.json
├── Makefile             # Development commands
└── README.md
```

**💡 Điểm hay của structure:**
- **Separation of Concerns:** Agents, tools, UI tách biệt
- **Modularity:** Dễ thêm agents/tools mới
- **Type Safety:** TypeScript + Python typing
- **Dev Experience:** Hot reload, linting, formatting

---

### 🧪 Bước 5: Tương tác và Customize

**Bây giờ bạn có thể:**
- 💬 **Chat với agent** qua web interface
- 🧪 **Test features** của fullstack application
- 🔧 **Xem code** trong `app/agents/` để học
- 🚀 **Customize agent** cho use case của bạn

**Sample Customization Tasks:**
```
1. Thêm tool mới vào agent (ví dụ: calculator, search)
2. Tạo agent mới cho domain khác
3. Customize UI theme và layout
4. Add authentication & user management
```

**💡 Tips:**
- Start từ example agents
- Copy-paste patterns từ workshop này
- Đọc ADK docs khi cần: https://github.com/google/adk-python
- Test thoroughly trước khi deploy

---

## 🎉 Hoàn Thành Bước 3!

**Bạn đã:**
- ✅ Setup fullstack agent application
- ✅ Chạy agent với professional UI
- ✅ Hiểu project structure và best practices
- ✅ Sẵn sàng build production apps!

**Tiếp theo:** Áp dụng vào hackathon workflow! 🚀

---

## 🎁 BONUS: Hackathon Power Tools

**Bây giờ bạn đã biết cách xây dựng agents và setup fullstack app, hãy áp dụng vào hackathon!**

### 🚀 Roadmap: MVP → Pitch

```
📋 MVP Planner  →  🎤 Pitch Generator
(Planning Agent)     (Pitch Agent)
      ↓                    ↓
 Project Brief       Landing Page
```

**Mục tiêu:** Trong 25 phút, bạn sẽ có:
- ✅ Project brief chi tiết cho MVP
- ✅ Pitch deck/landing page chuyên nghiệp

## 📋 Bonus 1: MVP Planning Agent (15 phút)

### Mục Tiêu
Biến ý tưởng thành **Project Brief** chi tiết để xây dựng MVP trong hackathon.

### Agent Workflow

```
💡 Hackathon Idea
     ↓
🎯 User Empathy Analysis
     ↓
✨ Feature Generation
     ↓
🔍 MVP Scope (Minimalism)
     ↓
📄 Project Brief → AI Studio Builder
```

### Tại Sao Cần Planning Agent?

**Vấn đề:** Ý tưởng hay nhưng:
- ❌ Scope quá rộng → không hoàn thành trong hackathon
- ❌ Không focus vào core value
- ❌ Over-engineering

**Giải pháp:** Planning Agent tự động:
- ✅ Phân tích user needs
- ✅ Generate features + prioritize
- ✅ Shortlist MVP scope (đủ để demo, không quá phức tạp)
- ✅ Tạo brief rõ ràng cho AI builders

---

In [14]:
# @title Tạo MVP Planning Agent

mvp_planning_agent = Agent(
    name="mvp_planning_agent",
    model=AGENT_MODEL,
    description="Phân tích ý tưởng hackathon và tạo MVP project brief chi tiết.",
    instruction="""Bạn là MVP Planning Expert cho hackathons.

NHIỆM VỤ:
1. Phân tích target users và pain points
2. Generate features và prioritize theo MOSCOW method
3. Tạo MVP brief tập trung vào core value

NGUYÊN TẮC MINIMALISM:
- MVP = Minimum **Viable** Product (đủ để validate, KHÔNG phải đầy đủ)
- Chỉ giữ lại features MUST-HAVE cho demo
- Đừng over-engineer: Simple > Complex trong hackathon
- Focus: 3-5 core features là đủ

WORKFLOW:
1. Nhận ý tưởng từ user
2. Sử dụng tư duy empathy để hiểu users
3. List những features tối thiểu thiết cho MVP
4. Tạo brief document

OUTPUT:
- Project Brief format chuẩn (Markdown)
- Sẵn sàng copy-paste vào AI Studio Builder/WebSim
- Rõ ràng, actionable, focused

QUAN TRỌNG:
- KHÔNG thêm quá nhiều features
- KHÔNG gợi ý tech stack phức tạp
- CÓ giải thích WHY cho mỗi must-have feature
"""
)

print_success(f"✅ MVP Planning Agent '{Colors.BOLD}{mvp_planning_agent.name}{Colors.ENDC}' sẵn sàng!")

✅ MVP Planning Agent 'mvp_planning_agent' sẵn sàng!


In [ ]:
# @title Test MVP Planning Agent

async def run_mvp_planning():
    print_header("\n🎬 TESTING MVP PLANNING AGENT")

    # Setup session
    session_service_mvp = InMemorySessionService()
    APP_NAME_MVP = "mvp_planner_demo"
    USER_ID_MVP = "user_planner"
    SESSION_ID_MVP = "session_planner_001"

    session_mvp = await session_service_mvp.create_session(
        app_name=APP_NAME_MVP,
        user_id=USER_ID_MVP,
        session_id=SESSION_ID_MVP
    )
    print_success(f"✅ Session: {Colors.BOLD}{SESSION_ID_MVP}{Colors.ENDC}\n")

    runner_mvp = Runner(
        agent=mvp_planning_agent,
        app_name=APP_NAME_MVP,
        session_service=session_service_mvp
    )

    # Test với ý tưởng từ Bonus 1
    test_idea = """
    Ý tưởng hackathon: "Study Buddy AI"

    Mô tả: Ứng dụng AI giúp sinh viên tìm partner học tập phù hợp dựa trên:
    - Môn học/subject
    - Learning style (visual, auditory, kinesthetic)
    - Schedule availability
    - Goals (pass exam, deep understanding, project)

    Sử dụng Gemini AI để match và gợi ý study plans.
    """

    await call_agent_async(
        f"Hãy phân tích ý tưởng này và tạo MVP brief chi tiết:\\n{test_idea}",
        runner=runner_mvp,
        user_id=USER_ID_MVP,
        session_id=SESSION_ID_MVP
    )

    print_header("🏁 KẾT THÚC TEST\n")
    print_info("📋 Copy brief ở trên và paste vào:")
    print("   → https://aistudio.google.com/apps")
    print("   → hoặc https://websim.ai")

await run_mvp_planning()

### 🚀 Sử Dụng Brief với AI Builders

**Workflow:**
1. ✅ Copy MVP Brief từ agent output
2. 🌐 Mở [Google AI Studio Apps](https://aistudio.google.com/apps)
3. 📝 Paste brief vào prompt
4. ⚡ AI Studio tự động generate:
   - React components
   - API endpoints
   - UI/UX design
   - Deployment config

**Alternative Builders:**
- [Builder.io](https://www.builder.io/) - Visual-first
- [Replit](https://replit.com/) - Full IDE
- [WebSim](https://websim.ai) - Instant prototypes

**💡 Pro Tip:** Brief càng chi tiết → App generated càng chính xác!

---

## 🎤 Bonus 2: Pitch Generator Agent (10 phút)

### Mục Tiêu
Tạo **Irresistible Pitch** biến judges từ skeptical → excited!

### The 4 Pillars of Winning Pitches

```
🎯 Extreme Clarity  → Judges hiểu ngay bạn làm gì
💎 Obvious Value    → ROI/Impact rõ ràng không cần giải thích
⚡ Low Friction    → Demo mượt, dễ imagine adoption
📦 Smart Packaging  → Every slide có job, không filler
```

### Pitch Structure

```
🎬 Hook (15s)         → "Imagine..."
💔 Problem (30s)      → Real pain + stats
✨ Solution (45s)     → Your product demo
📊 Traction (30s)     → Validation/MVP results  
🚀 Ask (15s)          → Clear next steps
```

**Total: 135 seconds = 2m15s** (perfect cho 3-5 min pitch với Q&A buffer)

---

In [16]:
# @title Tạo Pitch Generator Agent

pitch_generator_agent = Agent(
    name="pitch_generator_agent",
    model=AGENT_MODEL,
    description="Tạo irresistible hackathon pitch với 4 pillars: clarity, value, low friction, smart packaging.",
    instruction="""Bạn là Hackathon Pitch Expert - giúp teams thắng competitions.

4 PILLARS OF WINNING PITCHES:

1️⃣ **EXTREME CLARITY**
   - Judges hiểu product trong 10 giây
   - No jargon, no complexity
   - Format: "[Product] helps [User] do [Action] in [Timeframe]"

2️⃣ **OBVIOUS VALUE**
   - ROI rõ ràng (time saved, money saved, problem solved)
   - Use numbers & comparisons
   - Before/After contrast

3️⃣ **LOW FRICTION**
   - Demo phải mượt (no bugs visible!)
   - Judges có thể imagine adoption dễ dàng
   - Address objections proactively

4️⃣ **SMART PACKAGING**
   - Every slide có purpose
   - No filler content
   - Hook → Problem → Solution → Traction → Ask

WORKFLOW:
1. Nhận ý tưởng từ user
2. Create hook for Opening powerful
3. Define problem statement
4. Craft solution pitch
5. Generate pitch desk

OUTPUT FORMAT:
- Markdown structure (easy to copy)
- Ready for Gemini Canvas → HTML landing page
- Timing notes cho presenter
- Clear CTAs

REMEMBER:
- Judges see 20+ pitches → yours must POP
- Story > Features
- Passion > Polish (authenticity wins)
- Practice demo 10x before pitch
"""
)

print_success(f"✅ Pitch Generator Agent '{Colors.BOLD}{pitch_generator_agent.name}{Colors.ENDC}' sẵn sàng!")

✅ Pitch Generator Agent 'pitch_generator_agent' sẵn sàng!


In [ ]:
# @title Test Pitch Generator Agent

async def run_pitch_generator():
    print_header("\n🎬 TESTING PITCH GENERATOR AGENT")

    # Setup session
    session_service_pitch = InMemorySessionService()
    APP_NAME_PITCH = "pitch_generator_demo"
    USER_ID_PITCH = "user_pitcher"
    SESSION_ID_PITCH = "session_pitch_001"

    session_pitch = await session_service_pitch.create_session(
        app_name=APP_NAME_PITCH,
        user_id=USER_ID_PITCH,
        session_id=SESSION_ID_PITCH
    )
    print_success(f"✅ Session: {Colors.BOLD}{SESSION_ID_PITCH}{Colors.ENDC}\n")

    runner_pitch = Runner(
        agent=pitch_generator_agent,
        app_name=APP_NAME_PITCH,
        session_service=session_service_pitch
    )

    # Test với ý tưởng từ Bonus 1
    pitch_request = """
    Tạo pitch deck cho hackathon idea:

    **Study Buddy AI**
    Ứng dụng AI matching students với study partners phù hợp dựa trên:
    - Learning style
    - Subject/major
    - Schedule
    - Goals

    Target: University students in Vietnam
    Tech: Gemini AI + React + Firebase
    MVP đã có demo working

    Cần pitch trong 3 phút để thuyết phục judges.
    """

    await call_agent_async(
        pitch_request,
        runner=runner_pitch,
        user_id=USER_ID_PITCH,
        session_id=SESSION_ID_PITCH
    )

    print_header("🏁 KẾT THÚC TEST\n")
    print_info("🎨 Bước tiếp theo:")
    print("   1. Copy pitch content ở trên")
    print("   2. Mở Gemini hoặc Claude")
    print("   3. Prompt: 'Create an HTML landing page from this pitch:'")
    print("   4. Paste pitch content")
    print("   5. → Gemini generates beautiful landing page HTML!")
    print("   6. Save as index.html và deploy lên Vercel/Netlify")

await run_pitch_generator()

### 🎨 Sử Dụng Pitch với Gemini Canvas

**Workflow: Pitch → Landing Page**

1. **Copy pitch output** từ agent
2. **Open Gemini** (hoặc Claude Canvas)
3. **Prompt:**
```
Create a modern, professional HTML landing page for this hackathon project.

Use these design principles:
- Hero section với compelling headline
- Problem-Solution sections với visuals
- Demo video/screenshots placeholder
- Clear CTA buttons
- Responsive design (Tailwind CSS)
- Smooth animations

Here's the pitch content:
[PASTE YOUR PITCH]
```

4. **Gemini generates** complete HTML + CSS
5. **Download** và deploy lên:
   - [Vercel](https://vercel.com) (1-click)
   - [Netlify](https://netlify.com) (drag & drop)
   - [GitHub Pages](https://pages.github.com) (free)

**💡 Pro Tip:** Thêm demo video/GIF vào landing page → conversion rate ↑↑↑

---

---

## 🎊 Bonus Summary: Your Hackathon Toolkit

**Trong 25 phút, bạn đã có:**

### 📋 Bonus 1: MVP Planning Agent
- ✅ User empathy analysis
- ✅ Feature prioritization (MoSCoW)
- ✅ Project brief → AI Studio Builder
- ✅ Focus on minimalism & core value

### 🎤 Bonus 2: Pitch Generator Agent
- ✅ 4-pillar pitch framework
- ✅ Complete deck với timing
- ✅ Pitch content → Gemini Canvas → Landing page
- ✅ Ready to impress judges

---

## 🚀 Complete Workshop Flow

```
START
  ↓
🌤️ Bước 1: Single Agent với Tools
  ↓
👥 Bước 2: Multi-Agent Team với Delegation
  ↓
🚀 Bước 3: Fullstack App với ADK Starter Pack
  ↓
📋 Bonus 1: MVP Planning Agent → AI Studio → App
  ↓
🎤 Bonus 2: Pitch Generator → Gemini Canvas → Landing Page
  ↓
🏆 BUILD & WIN!
```

**Total Time:**
- Core Workshop: 60 min (Agent basics + Multi-agent + ADK Starter Pack)
- Bonus: 25 min (MVP Planning + Pitch Generation)
- **= 85-90 minutes complete workshop!**

---

## 💪 Vibe Coding Challenge: Your Turn!

**Nhiệm vụ cuối:**
1. ✅ Customize agent từ Starter Pack cho ý tưởng của bạn
2. ✅ Run qua Planning & Pitch agents (Bonus 1 & 2)
3. ✅ Deploy MVP lên Vercel/Replit
4. ✅ Publish landing page
5. ✅ Share với group!

**Deadline:** Before end of workshop!

**Prize:** Best project wins bragging rights + potential mentorship! 🏆

---

## 🎓 Final Thoughts

### Quote to Remember

> **"Your job is no longer to write code. Your job is to write the story — the WHY and WHAT — so clearly that AI will handle the HOW. Focus on orchestrating outcomes, not obsessing over implementation details."**

### Key Learnings

**From Today's Workshop:**
- 🤖 **Bước 1:** Built single agent with tools
- 👥 **Bước 2:** Created multi-agent teams with delegation
- 🚀 **Bước 3:** Setup fullstack app với ADK Starter Pack
- 📋 **Bonus 1:** MVP Planning Agent cho hackathons
- 🎤 **Bonus 2:** Pitch Generator Agent cho presentations
- ⚡ **Applied:** Real hackathon workflow from idea to pitch

**What's Next:**
- Explore more [MCP servers](https://github.com/modelcontextprotocol/servers)
- Build complex multi-agent systems
- Contribute to [ADK community](https://github.com/google/adk-python)
- Share your hackathon wins!

### Resources

**Documentation:**
- [ADK Python Docs](https://github.com/google/adk-python)
- [Agent Starter Pack](https://github.com/GoogleCloudPlatform/agent-starter-pack)
- [Google AI Studio](https://aistudio.google.com)

**Tools Mentioned:**
- [AI Studio Apps Builder](https://aistudio.google.com/apps)
- [Builder.io](https://www.builder.io)
- [Replit](https://replit.com)

---

## 🙏 Thank You!

**Cảm ơn các bạn đã tham gia workshop!**

Stay connected:
- Workshop repo: [Github](https://github.com/chitoan1992/devfest-google-adk)
- Community: [GDG Miền Trung](https://www.facebook.com/Gdgmientrung/)
- Feedback form: [Link](https://app.formbricks.com/s/cmgv2723h05paad018xlyabgq)

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/You%20complete%20me.svg)

**Remember:** The best way to learn is to build. Go create something amazing! 🚀

---

**🤖 Built with Google ADK | GDG DevFest Đà Nẵng 2025**